# MMA 662: Decision Analytics
# Assignment 1 on LP: The Story of an Imbalance!

> "The balance must be maintained. That is why I have returned." -Thanos


In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np

***Fresh Milk Transportation***

*BalancedMilk* is an old dairy-transportation firm known for the freshness of the milk it distributes. It has long-term fixed-amount contracts with eight dairy farms (suppliers) and distributes their production among ten demand markets. Transporting milk for half a century from suppliers to demand markets, BalancedMilk is well aware of the milk demand in each demand market and the transportation cost from each supplier to each demand center.

In the following table, the number in front of each supply and demand center represents the supply and demand (in tonnes.) Also, for each supply-demand pair, the transportation cost (dollars per tonne) is depicted.

| | $D_1$(90)| $D_2$(100) | $D_3$(150) | $D_4$(190) | $D_5$(180) | $D_6$(240) | $D_7$(210) | $D_8$(90) | $D_9$(160) | $D_{10}$(70) |
|--| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| $S_1$(110) |20|49|16|30|8|35|21|40|10|12|
| $S_2$(80) |15|53|7|20|47|11|16|17|15|44|
| $S_3$(100) |22|25|42|22|31|9|11|29|20|5|
| $S_4$(240) |45|6|33|35|49|25|30|47|32|31|
| $S_5$(100) |9|12|41|15|38|14|53|22|12|13|
| $S_6$(280) |21|24|32|49|5|47|30|23|37|8|
| $S_7$(130) |12|19|5|28|47|39|15|35|9|51|
| $S_8$(440) |34|17|10|21|9|33|14|26|19|45|

For example, transporting one tonne of milk from supply center $S_1$ to demand market $D_1$ costs 20 dollars.

**Note**: In the all of the follwoing problems, the firm cannot supply milk to a demand market **more** than its demand, i.e., cannot **over-supply** it. 

## Problem 1: (2.5 pts)

The firm wants to **distribute all the supplied milk as efficiently as possible**. We wrote a code that estimates the optimal milk distribution and the total transportation cost. Run the code and find the optimal milk distribution and the total transportation cost. Report the total transportation cost in dollars.

In [67]:
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 440]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

#######################
# Optimization Problem:
#######################

model = gp.Model("BalancedMilk_1")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total distribution cost:
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:

if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")

Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 

Solution status is optimal, and the minimum cost is: $18440. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 20.0
Supply from S_2 to D_8 : 60.0
Supply from S_3 to D_6 : 100.0
Supply from S_4 to D_2 : 100.0
Supply from S_4 to D_6 : 140.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_4 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_8 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 80.0
Supply from S_7 to D_9 : 50.0
Supply from S_8 to D_3 : 50.0
Supply from S_8 to D_4 : 180.0
Supply from S_8 to D_7 : 210.0


D_1 is experiencing a supply shortage of 0.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 0.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 0.0 tonnes.
D_7 is experiencing a supply shortage of 0.0 tonnes.
D_8 is experiencing a supply shortage of 0.0 tonnes.
D_9 is experiencing a s

## The incident!
Due to a cattle virus outbreak in the largest supplier ($S_8$), the Ministry of Health shut down this dairy farm. Left with seven suppliers and unchanged demand, BalancedMilk is facing a supply shortage. Therefore, the management establishes two teams to search for short-term and long-term strategies to combat this issue. While the *long-term team* is looking for alternative suppliers or new contracts, we want to evaluate the strategies the *short-term team* offers in the following problems. In all of the following problems, there is no milk supply from $S_8$.

**Note**: Before the shutdown of $S_8$, the total supply and demand were equal - thereby a "balanced" situation. You are familiar with balanced problems; now, we enter the imbalanced realm! 

## Problem 2: (7.5 pts)

Suppose not supplying milk to a demand market is costless. Nevertheless, **the firm must distribute all the milk the remaining seven supply centers provide**. Also, remember that **the firm cannot oversupply a demand market**. 

Use the code for the previous part and adjust it for the new situation to get the optimal allocation and cost. Report the optimal cost. Hint: There are many ways to do this, but the simplest way is to adjust the supply vector!

In [68]:
## Your code for Problem 2 goes here:
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total distribution cost:
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:

if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")

Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 

Solution status is optimal, and the minimum cost is: $10670. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 60.0
Supply from S_2 to D_6 : 20.0
Supply from S_3 to D_6 : 80.0
Supply from S_3 to D_7 : 20.0
Supply from S_4 to D_2 : 100.0
Supply from S_4 to D_6 : 140.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_9 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_8 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 90.0
Supply from S_7 to D_9 : 40.0


D_1 is experiencing a supply shortage of 0.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 190.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 0.0 tonnes.
D_7 is experiencing a supply shortage of 190.0 tonnes.
D_8 is experiencing a supply shortage of 60.0 tonnes.
D_9 is experiencing a supply shortage of 0.0 tonnes.
D_10 is experiencing a suppl

## Problem 3: (5 pts)

In the real world, with ruthless competitors ready to attack *BalancedMilk*'s market share, the cost of not supplying milk to a demand center is not zero. Suppose the cost of not delivering each tonne of milk to each demand center is 20 dollars/tonne. Adjust the code you wrote for Problem 2 to give you the optimal cost and allocation. Report the optimal cost. (Remember, like Problem 2, the firm should distribute all the milk provided by the remaining seven supply centers and cannot oversupply a demand market.)

Hint-1: One way to implement this is to adjust the objective function. Hint-2: If done correctly, the new optimal cost must be \$19470.

In [39]:
## Your code for Problem 3 goes here:


Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

cost_not_supply = (sum(Demand[j] for j in Range_demand)-(sum(Supply[i] for i in Range_supply)))*20


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total cost(supplying and not supplying):
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand) + cost_not_supply
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:
supplied=0
if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
           supplied=supplied+v.x
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")
print("Total milk supplied is: ",supplied)
print("\n")

Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 

Solution status is optimal, and the minimum cost is: $19470. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 60.0
Supply from S_2 to D_6 : 20.0
Supply from S_3 to D_6 : 80.0
Supply from S_3 to D_7 : 20.0
Supply from S_4 to D_2 : 100.0
Supply from S_4 to D_6 : 140.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_9 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_8 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 90.0
Supply from S_7 to D_9 : 40.0


Total milk supplied is:  1040.0


D_1 is experiencing a supply shortage of 0.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 190.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 0.0 tonnes.
D_7 is experiencing a supply shortage of 190.0 tonnes.
D_8 is experiencing a supply shortage of 60.0 tonnes.
D_9 is experiencing a supply shortage of 0.0 to

## Problem 4: (5 pts)

Now, suppose the cost of not delivering each tonne of milk to each demand center is 100 dollars/tonne instead of 20 dollars/tonne. Adjust your code from Problem 3 for the new situation to give you the optimal cost and allocation. (Remember, like Problem 2, the firm should distribute all the milk provided by the remaining seven supply centers and cannot oversupply a demand market.) Report the optimal cost.


In [9]:
## Your code for Problem 3 goes here:


Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

cost_not_supply = (sum(Demand[j] for j in Range_demand)-(sum(Supply[i] for i in Range_supply)))*100


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total cost(supplying and not supplying):
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand) + cost_not_supply
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:

if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")

Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 

Solution status is optimal, and the minimum cost is: $54670. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 60.0
Supply from S_2 to D_6 : 20.0
Supply from S_3 to D_6 : 80.0
Supply from S_3 to D_7 : 20.0
Supply from S_4 to D_2 : 100.0
Supply from S_4 to D_6 : 140.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_9 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_8 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 90.0
Supply from S_7 to D_9 : 40.0


D_1 is experiencing a supply shortage of 0.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 190.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 0.0 tonnes.
D_7 is experiencing a supply shortage of 190.0 tonnes.
D_8 is experiencing a supply shortage of 60.0 tonnes.
D_9 is experiencing a supply shortage of 0.0 tonnes.
D_10 is experiencing a suppl

## Problem 5: (10 pts)

You have the optimal allocations for Problems 2, 3, and 4. Are they different? Are they similar? Why? Justify your answer.


Your explanation for Problem 5: ...

-----------------------------------------------------------------------------------------------------------------------------------------------
Ans:
Yes all the problems are similar. Since the firm has to supply all the milk available at the supply and they are short of 440 Tonnes of milk in all the three cases because of non availability of S8 centre. So we see that a constant cost has been added which is not affecting the decision variables(X)at all.
The optimal cost in case 1 is: 10670 $

Without even running the optimisation, optimal cost can be find in 3d and 4th case.

Optimal cost in 3rd case = The optimal cost in case 1 + cost of short supply = 10670 + 20*440 = 19470 $

Optimal cost in 3rd case = The optimal cost in case 1 + cost of short supply = 10670 + 100*440 = 54670 $


## Problem 6.1: (5 pts)

Until now, we have assumed the firm is committed to distributing all the milk provided by the suppliers. Now, suppose the firm is not committed to distributing all the milk provided by the suppliers. In other words, the firm can choose not to distribute some of the milk provided by the suppliers. Suppose the cost of not distributing each tonne of milk is 20 dollars/tonne. Adjust your code from Problem 3 for the new situation to give you the optimal allocation and cost. Report the optimal cost.

In [37]:
## Your code for Problem 6.1 goes here:

Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]

DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

total_supply=sum(Supply[i] for i in range(len(Supply)))
total_demand=sum(Demand[i] for i in range(len(Demand)))

cost_not_distributed=20 #per tonne for snot distributing the milk
cost_demand_not_fulfilled=0 # per tonne for shortage in demand

n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total cost(supplying and not supplying):
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)+(cost_not_distributed*(total_supply-sum(X[i,j]for i in Range_supply for j in Range_demand)))+(cost_demand_not_fulfilled*(total_demand-sum(X[i,j]for i in Range_supply for j in Range_demand)))
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) <= Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:

supplied=0
if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
           supplied=supplied+v.x
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")
print("Total Demand          : ",total_demand)
print("Total Supply          : ",total_supply)
print("Supplied              : ",supplied)
print("Demand not fulfilled  : ",total_demand-sum(Solution[i,j] for i in Range_supply for j in Range_demand))
print("Supply not distributed: ",total_supply-sum(Solution[i,j] for i in Range_supply for j in Range_demand))


print("\n")
Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply)))
  
print("\n")

for i in Range_supply:
  print(SupplyCenter[i], 
        "is experiencing a supply surplus of {} tonnes.".format(Supply[i] - sum(Solution[i, j] for j in Range_demand)))
  



Solution status is optimal, and the minimum cost is: $9840. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 20.0
Supply from S_2 to D_6 : 60.0
Supply from S_3 to D_6 : 100.0
Supply from S_4 to D_2 : 100.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_9 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 130.0


Total Demand          :  1480
Total Supply          :  1040
Supplied              :  870.0
Demand not fulfilled  :  440.0
Supply not distributed:  0.0


D_1 is experiencing a supply shortage of 0.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 190.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 80.0 tonnes.
D_7 is experiencing a supply shortage of 210.0 tonnes.
D_8 is experiencing a supply shortage of 90.0 tonnes.
D_9 is experiencing a supply shortage of 40.0 t

## Problem 6.2: (7.5 pts)

Compare the optimal allocations for Problem 6.1 and Problem 3. Hint: They are different!

Why? Explain the reason behind this difference.

Your explanation for Problem 6.2: 
In Problem 3 we have supplied all the milk which is available for supply i.e 1040 Tonnes but in problem 6.1 since we can store the milk at 20$ per tonne So our model optimised the problem by not supplying in cases where the costs of supplying is more than storing the milk. In optimal solution we have supplied only 870 Tonnes and hold 170 Tonnes of milk.

## Problem 7.1: (5 pts)

Suppose the firm is not committed to distributing all the milk provided by the suppliers. In other words, the firm can choose not to distribute some of the milk provided by the suppliers. Suppose the cost of not distributing each tonne of milk is 100 dollars/tonne instead of 20 dollars/tonne. Adjust your code from Problem 4 for the new situation to give you the optimal allocation and cost. Report the optimal cost.

In [42]:
## Your code for Problem 7.1 goes here:

Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]

DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

total_supply=sum(Supply[i] for i in range(len(Supply)))
total_demand=sum(Demand[i] for i in range(len(Demand)))

cost_not_distributed=100 #per tonne for snot distributing the milk
cost_demand_not_fulfilled=0 # per tonne for shortage in demand

n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total cost(supplying and not supplying):
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)+(cost_not_distributed*(total_supply-sum(X[i,j]for i in Range_supply for j in Range_demand)))+(cost_demand_not_fulfilled*(total_demand-sum(X[i,j]for i in Range_supply for j in Range_demand)))
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) <= Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:
supplied=0
if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
           supplied=supplied+v.x
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")
print("Total Demand          : ",total_demand)
print("Total Supply          : ",total_supply)
print("Supplied              : ",supplied)
print("Demand not fulfilled  : ",total_demand-sum(Solution[i,j] for i in Range_supply for j in Range_demand))
print("Supply not distributed: ",total_supply-sum(Solution[i,j] for i in Range_supply for j in Range_demand))


print("\n")
Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply)))
  
print("\n")

for i in Range_supply:
  print(SupplyCenter[i], 
        "is experiencing a supply surplus of {} tonnes.".format(Supply[i] - sum(Solution[i, j] for j in Range_demand))) 

Solution status is optimal, and the minimum cost is: $10670. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 60.0
Supply from S_2 to D_6 : 20.0
Supply from S_3 to D_6 : 80.0
Supply from S_3 to D_7 : 20.0
Supply from S_4 to D_2 : 100.0
Supply from S_4 to D_6 : 140.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_9 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_8 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 90.0
Supply from S_7 to D_9 : 40.0


Total Demand          :  1480
Total Supply          :  1040
Supplied              :  1040.0
Demand not fulfilled  :  440.0
Supply not distributed:  0.0


D_1 is experiencing a supply shortage of 0.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 190.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 0.0 tonnes.
D_7 is experiencing a supply shortag

## Problem 7.2: (7.5 pts)

Compare the optimal allocations for Problem 7.1 and Problem 4. Are they different? Are they the same? Why? Explain your answer.

Your explanation for Problem 7.2: They are same. Because the cost of holding the milk is so high. The model optimized never considered the case of not distributing the milk because it is always cheaper to distribute the milk even at max distribution cost of 49$ per tonne in comparison to 100$ of not distributing cost.

## Problem 8.1: (10 pts)

We have assumed the no-supply cost is the same for all demand centers until now. However, this may not be the case. The estimations by the *short-term team* indicate that the cost of not supplying a tone of milk to each demand center is as follows:

| | $D_1$| $D_2$ | $D_3$ | $D_4$ | $D_5$ | $D_6$ | $D_7$ | $D_8$ | $D_9$ | $D_{10}$|
|--| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| Cost of not supplying (dollars/tonne) |15|20|25|30|28|35|32|15|25|10|

Additionally, like Problems 2, 3, and 4, the firm should distribute all the milk provided by the remaining seven supply centers. Use your code for Problems 2 or 3 or 4, and adjust it for the new situation. Get the optimal allocation and the optimal cost. Report the optimal cost.

Hint: One way to do this is to adjust the objective function. 


In [43]:
## Your code for Problem 8.1 goes here:

Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]

DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

total_supply=sum(Supply[i] for i in range(len(Supply)))
total_demand=sum(Demand[i] for i in range(len(Demand)))

cost_not_distributed=0 #per tonne for snot distributing the milk
cost_demand_not_fulfilled=[15,20,25,30,28,35,32,15,25,10] # per tonne for shortage in demand

n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total cost(supplying and not supplying):
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)+(cost_not_distributed*(total_supply-sum(X[i,j]for i in Range_supply for j in Range_demand)))+sum((cost_demand_not_fulfilled[j]*(Demand[j]-sum(X[i,j] for i in Range_supply)))for j in Range_demand)
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:
supplied=0
if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
           supplied=supplied+v.x
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")
print("Total Demand          : ",total_demand)
print("Total Supply          : ",total_supply)
print("Supplied              : ",supplied)
print("Demand not fulfilled  : ",total_demand-sum(Solution[i,j] for i in Range_supply for j in Range_demand))
print("Supply not distributed: ",total_supply-sum(Solution[i,j] for i in Range_supply for j in Range_demand))


print("\n")
Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply)))
  
print("\n")

for i in Range_supply:
  print(SupplyCenter[i], 
        "is experiencing a supply surplus of {} tonnes.".format(Supply[i] - sum(Solution[i, j] for j in Range_demand))) 

Solution status is optimal, and the minimum cost is: $21980. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 20.0
Supply from S_2 to D_6 : 60.0
Supply from S_3 to D_7 : 100.0
Supply from S_4 to D_2 : 100.0
Supply from S_4 to D_6 : 140.0
Supply from S_5 to D_4 : 60.0
Supply from S_5 to D_6 : 40.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_7 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 130.0


Total Demand          :  1480
Total Supply          :  1040
Supplied              :  1040.0
Demand not fulfilled  :  440.0
Supply not distributed:  0.0


D_1 is experiencing a supply shortage of 90.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 130.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 0.0 tonnes.
D_7 is experiencing a supply shortage of 80.0 tonnes.
D_8 is experiencing a supply shortage o

## Problem 8.2: (10 pts)

Compare the optimal allocation for Problem 8.1 and Problems 2, 3, or 4. Are they different? Are they the same? Why? Explain your answer.

Your explanation for Problem 8.2: Optimal allocation is different from Problems 2,3, or 4 because objective function which to minimize the cost is dependent on allocation which was not the case in problems 2,3 or 4 where no supply cost is same for each demand centre. But in problem 8.2 with different costs associated with each demand centre we get a new allocation which minimised the objective function.

## Problem 9.1: (5 pts)

Suppose not supplying milk to a demand market is costless, and the firm is committed to supplying all the available milk - like Problem 2. But this time, the firm decides to supply the milk so that each demand center receives at least 50% of its demand. Use the code for Problem 2 and adjust it for the new situation to get the optimal allocation and cost. Report the optimal cost.

In [64]:
## Your code for Problem 9.1 goes here:
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total distribution cost:
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])


# 3. Demand constraints :
# For each demand market, the amount of milk demanded from this market should met 50% of its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) >= 0.5*Demand[j], "Demand_"+DemandMarket[j])


# Optimization
model.optimize()

# Printing the results:
supplied=0
if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
           supplied=supplied+v.x
else:
    print('Optimization was stopped with status %d' % model.status)
print("\n")
print("Total milk supplied is: ",supplied)
print("\n")

Solution = model.getAttr('x', X) 
shortage=0
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
    print(DemandMarket[j], "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 
    shortage=shortage+Demand[j] - sum(Solution[i, j] for i in Range_supply)
print("\n")
print("Total shortage is: ",shortage)
print("\n")

not_supplied=0
for i in Range_supply:
    print(SupplyCenter[i], "is experiencing a supply not distributed:  {} tonnes.".format(Supply[i] - sum(Solution[i, j] for j in Range_demand)))
    not_supplied=not_supplied+Supply[i] - sum(Solution[i, j] for j in Range_demand)
print("\n")
print("Total not supplied is: ",not_supplied)

Solution status is optimal, and the minimum cost is: $11605. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 65.0
Supply from S_2 to D_8 : 15.0
Supply from S_3 to D_7 : 100.0
Supply from S_4 to D_2 : 100.0
Supply from S_4 to D_6 : 135.0
Supply from S_4 to D_7 : 5.0
Supply from S_5 to D_1 : 5.0
Supply from S_5 to D_4 : 95.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_8 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_1 : 40.0
Supply from S_7 to D_3 : 85.0
Supply from S_7 to D_9 : 5.0


Total milk supplied is:  1040.0


D_1 is experiencing a supply shortage of 45.0 tonnes.
D_2 is experiencing a supply shortage of 0.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 95.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 105.0 tonnes.
D_7 is experiencing a supply shortage of 105.0 tonnes.
D_8 is experiencing a supply shortage of 45.0 tonnes.
D_9 is experienci

## Problem 9.2: (7.5 pts)

Compare the optimal cost with your answer to Problem 2. Is it more, or is it less than the optimal cost for Problem 2? Why?

Your explanation for Problem 9.2: optimal cost in problem 9.2 which is 11605$ is more than 10670$(optimal cost in problem 2). It is the cost we are paying to fulfill atleast 50% demand to all the Demand centres and in doing so we are sending less milk to the demand centres where distribution cost is low and sending to the deman centres where costs are high to meet the constraint of 50% fulfillment.
Eg: s_3 to D_7 become 100 Tonnes in problem 9.2 from 420 Tonnes in problem 2. This demand must be come from such S->D where cost of distribution should be higher than cost of distribution of S_3->D_7

## Prolem 9.3: (7.5 pts)

Suppose the firm sets this bar to 90%. Every demand center must receive at least 90% of its demand. Is it possible? Why? What is the maximum possible percentage?

Your answer for Problem 9.3: No it is not possible at 90% total supply required is 1332 Tonnes but we have only 1040 Tonnes of supply. The maximum % we can do is ~70.27% since we have 1040 Tonnes of supply available and 1480 tonnes of demand.

In [61]:

Supply = [110, 80, 100, 240, 100, 280, 130]
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

total_supply=sum(Supply[i] for i in range(len(Supply)))
total_demand=sum(Demand[i] for i in range(len(Demand)))

print("total_supply :",total_supply)
print("total_demand :",total_demand)

print("total demand at 90% : ",total_demand*0.9)

print("total demand available: ", total_supply/total_demand*100,"%")



total_supply : 1040
total_demand : 1480
total demand at 90% :  1332.0
total demand available:  70.27027027027027 %


## Sensitivity Analysis

(To obtain the sensitivity analysis for each problem, use the following code to give you the optimal cost and the sensitivity analysis instead of the optimal allocation.)

In [65]:
# Printing the results: 

if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g.' % model.objVal)
#    for v in model.getVars():
#        if v.x > 0:
#            print(v.varName,':', v.x,)
   header = "| {:<15} | {:<6} | {:<6} | {:<6} | {:<6} | {:<10} | {:<10} |".format("Name", "Sense", "Slack", "Shadow Price", "RHS", "SARHSLow", "SARHSUp")
   print(header)
   print("-" * len(header))  # Print a separator line
   for constr in model.getConstrs():
       name = constr.ConstrName
       sense = constr.Sense
       slack = constr.Slack
       pi_val = constr.Pi
       rhs = constr.RHS
       sarhslow = constr.SARHSLow
       sarhsup = constr.SARHSUp

       row = "| {:<15} | {:<6} | {:<6.2f} | {:<6.2f} | {:<6.2f} | {:<10.2f} | {:<10.2f} |".format(name, sense, slack, pi_val, rhs, sarhslow, sarhsup)
       print(row)
else:
    print('Optimization was stopped with status %d' % model.status)

Solution status is optimal, and the minimum cost is: $11605.
| Name            | Sense  | Slack  | Shadow Price | RHS    | SARHSLow   | SARHSUp    |
---------------------------------------------------------------------------------------
| Supply_S_1      | =      | 0.00   | 10.00  | 110.00 | 75.00      | 155.00     |
| Supply_S_2      | =      | 0.00   | 11.00  | 80.00  | 75.00      | 125.00     |
| Supply_S_3      | =      | 0.00   | 6.00   | 100.00 | 85.00      | 105.00     |
| Supply_S_4      | =      | 0.00   | 25.00  | 240.00 | 225.00     | 345.00     |
| Supply_S_5      | =      | 0.00   | 6.00   | 100.00 | 95.00      | 140.00     |
| Supply_S_6      | =      | 0.00   | 17.00  | 280.00 | 275.00     | 295.00     |
| Supply_S_7      | =      | 0.00   | 9.00   | 130.00 | 125.00     | 175.00     |
| Demand_D_1      | <      | 45.00  | 0.00   | 90.00  | 45.00      | inf        |
| Demand_D_2      | <      | 0.00   | -19.00 | 100.00 | 50.00      | 115.00     |
| Demand_D_3      | <    

Explanations:
1. Sense: the nature of the constraint. If it is >=, <=, or ==.
2. RHS: the right-hand side of the constraint.
3. Slack: The gap between the left-hand side of a constraint and the RHS in the optimal point.
4. Shadow Price: Please refer to the slides for the definition of the shadow price.
5. SARHSLow: The lowest value of the RHS for which we can still use this sensitivity analysis table to estimate the optimal cost, and we do not need to re-optimize the problem to estimate the optimal cost.
6. SARHSUp: The highest value of the RHS for which we can still use this sensitivity analysis table to estimate the optimal cost, and we do not need to re-optimize the problem to estimate the optimal cost.

(SARHSLow and SARHSUp give us the allowable range of the RHS for which the geometry of the optimal solution remains unchanged.)

## Problem 10.1: (15 pts)

Rewrite the solution for Problem 2 here, but replace the last part with the code above to get the sensitivity analysis. If done correctly, the shadow price for all supply centers is positive. Now, answer the following questions.

1. If we increase the supply of $S_1$ by 10 tonnes from 110 to 120, how much would the optimal cost change?

2. Suppose we want to estimate the optimal cost without re-optimizing the problem. How much can we increase or decrease the supply of $S_1$ to calculate the optimal cost without re-optimizing the problem?

3. Explain what a positive shadow price in a minimization problem means, then explain why in Problem 2 the shadow price for all supply centers is positive.

In [73]:
## Your code for Problem 10.1 goes here:
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 1040, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)


model = gp.Model("BalancedMilk_2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total distribution cost:
exp = gp.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjective(exp, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:
supplied=0
if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
           supplied=supplied+v.x
else:
    print('Optimization was stopped with status %d' % model.status)
print("\n")
print("Total milk supplied is: ",supplied)
print("\n")

Solution = model.getAttr('x', X) 
shortage=0
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
    print(DemandMarket[j], "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 
    shortage=shortage+Demand[j] - sum(Solution[i, j] for i in Range_supply)
print("\n")
print("Total shortage is: ",shortage)
print("\n")

not_supplied=0
for i in Range_supply:
    print(SupplyCenter[i], "is experiencing a supply not distributed:  {} tonnes.".format(Supply[i] - sum(Solution[i, j] for j in Range_demand)))
    not_supplied=not_supplied+Supply[i] - sum(Solution[i, j] for j in Range_demand)
print("\n")
print("Total not supplied is: ",not_supplied)


# Printing the results: 

if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g.' % model.objVal)
#    for v in model.getVars():
#        if v.x > 0:
#            print(v.varName,':', v.x,)
   header = "| {:<15} | {:<6} | {:<6} | {:<6} | {:<6} | {:<10} | {:<10} |".format("Name", "Sense", "Slack", "Shadow Price", "RHS", "SARHSLow", "SARHSUp")
   print(header)
   print("-" * len(header))  # Print a separator line
   for constr in model.getConstrs():
       name = constr.ConstrName
       sense = constr.Sense
       slack = constr.Slack
       pi_val = constr.Pi
       rhs = constr.RHS
       sarhslow = constr.SARHSLow
       sarhsup = constr.SARHSUp

       row = "| {:<15} | {:<6} | {:<6.2f} | {:<6.2f} | {:<6.2f} | {:<10.2f} | {:<10.2f} |".format(name, sense, slack, pi_val, rhs, sarhslow, sarhsup)
       print(row)
else:
    print('Optimization was stopped with status %d' % model.status)

Solution status is optimal, and the minimum cost is: $7970. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_3 : 20.0
Supply from S_2 to D_6 : 60.0
Supply from S_3 to D_6 : 100.0
Supply from S_4 to D_2 : 240.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_2 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_8 : 30.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 130.0


Total milk supplied is:  1040.0


D_1 is experiencing a supply shortage of 0.0 tonnes.
D_2 is experiencing a supply shortage of 790.0 tonnes.
D_3 is experiencing a supply shortage of 0.0 tonnes.
D_4 is experiencing a supply shortage of 190.0 tonnes.
D_5 is experiencing a supply shortage of 0.0 tonnes.
D_6 is experiencing a supply shortage of 80.0 tonnes.
D_7 is experiencing a supply shortage of 210.0 tonnes.
D_8 is experiencing a supply shortage of 60.0 tonnes.
D_9 is experiencing a supply shortage of 50.0 tonnes.
D_10 is experiencing a supply shortage of 0.0 tonnes.


Total shortage is:  1380

Your explanation for Problem 10.1: 
1. 12*10=120$ increase

2. we can fluctuate S1 supply between 90 and 150 Tonnes i.e from 110 we can decrease supply by 20Tonnes or increase by 40 Tonnes and still calculate the optimal cost without optimizing the problem again.

3. Positive shadow price in a minimisation problem mean that an increase in the right-hand side value of a constraint by x unit will lead to an increase in the objective function value by x*shadow price. Here all the shadow prices are positive because we have to distribute all the supply vailable to us to fulfill the demand and for each tonne of supply increase there is a cost associated to distribute the same and hence it would not be possible to have negative shadow pries in this case. Eg: if I get 1 tonne extra supply then my cost will increase as I cannot distribute this additional supply for free, I have to bear a cost to distribute this extra milk supply to a demand centre.

## Problem 10.2: (10 pts)

In the settings of Problem 2, what is even more interesting than positive shadow prices for all supply centers is that the shadow price for some demand markets is negative! How is it possible that increasing the demand for a demand market decreases the optimal cost? 

Hint: If you want to gain some intuition, you can change the demand for $D_2$ by a small amount and see what happens to the slack variable for $D_4$, $D_7$ and $D_8$.


Your explanation for Problem 10.2: 

Distribution of supply to some demand markets are lower than that of the other markets. Hence if we increase the demand in markets with 0 slack (they happen to be the one with negative shadow price) than they can absorb more supply which is currently going to the demand markets where the distribution cost is more.

****